In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

print("Imports Complete")

Imports Complete


In [53]:
df = pd.read_csv("rounds_joined.csv")
df1 = df.copy()

# Make the last column the target (winner of the round)
df1['target'] = df1['Team1_Won']

# Drop the winner column
df1 = df1.drop(columns=['Team1_Won', 'Team2_Won', 'Round_Number', 'Tournament_Name', 'Source_File', 'Team1_Code', 'Team2_Code', 'Team1_Member1_Name', 'Team1_Member2_Name', 'Team2_Member1_Name', 'Team2_Member2_Name', 'Team1_Member1_Points', 'Team1_Member2_Points', 'Team2_Member1_Points', 'Team2_Member2_Points'])

# For Team1_Side and Team2_Side, replace all instances of 'Aff' with 1 and 'Neg' with 0.
df1['Team1_Side'] = df1['Team1_Side'].replace({'Aff': 1, 'Neg': 0})
df1['Team2_Side'] = df1['Team2_Side'].replace({'Aff': 1, 'Neg': 0})

# DATA INSPECTION AND CLEANING
print("Dataset shape:", df1.shape)
print("\nMissing values per column:")
print(df1.isnull().sum())
print("\nData types:")
print(df1.dtypes)
print("\nInfinite values check:")
print(np.isinf(df1.select_dtypes(include=[np.number])).sum())

# Handle missing values - DROP ROWS with missing data
print(f"Before dropping: {len(df1)} rows")
df1 = df1.dropna()
print(f"After dropping missing values: {len(df1)} rows")
print(f"Rows dropped: {len(df.copy()) - len(df1)}")

# Handle any remaining infinite values
df1 = df1.replace([np.inf, -np.inf], np.nan)
df1 = df1.dropna()  # Drop any rows with infinite values too

# ALTERNATIVE: Fill missing values with median (commented out)
# Uncomment the lines below and comment out the dropna() approach above to switch back
# numeric_columns = df1.select_dtypes(include=[np.number]).columns
# df1[numeric_columns] = df1[numeric_columns].fillna(df1[numeric_columns].median())
# df1 = df1.replace([np.inf, -np.inf], np.nan)
# df1 = df1.fillna(0)

print("\nAfter cleaning - Missing values:")
print(df1.isnull().sum().sum())
print("After cleaning - Infinite values:")
print(np.isinf(df1.select_dtypes(include=[np.number])).sum().sum())


# Determine null model



Dataset shape: (3392, 38)

Missing values per column:
Year                                 0
Team1_Side                           0
Team1_Member1_Rank                   0
Team1_Member2_Rank                   0
Team2_Side                           0
Team2_Member1_Rank                   0
Team2_Member2_Rank                   0
Team1_Total_Wins                    98
Team1_Total_Losses                  98
Team1_Prelim_Wins                   98
Team1_Prelim_Losses                 98
Team1_Num_Tournaments               98
Team1_Rank_Points                   98
Team1_National_Rank                 98
Team1_State_Rank                    98
Team1_Win_Rate                      98
Team1_Prelim_Win_Rate               98
Team1_Total_Rounds                  98
Team1_Avg_Points_Per_Tournament     98
Team1_National_Exposure            188
Team1_Avg_Tournament_Size          188
Team1_Tournament_Points            188
Team2_Total_Wins                   125
Team2_Total_Losses                 125
Team2_Prel

/var/folders/nr/k2xcn76959l8v4z4shd8k5sh0000gn/T/ipykernel_29422/3999815525.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['Team1_Side'] = df1['Team1_Side'].replace({'Aff': 1, 'Neg': 0})
/var/folders/nr/k2xcn76959l8v4z4shd8k5sh0000gn/T/ipykernel_29422/3999815525.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['Team2_Side'] = df1['Team2_Side'].replace({'Aff': 1, 'Neg': 0})


In [54]:
# FEATURE NORMALIZATION - Very important for neural networks!
from sklearn.preprocessing import StandardScaler

# Separate features and target
X = df1.iloc[:, :-1]
y = df1.iloc[:, -1]

# Normalize features to have mean=0 and std=1
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Feature statistics after scaling:")
print(f"Mean: {X_scaled.mean():.6f}")
print(f"Std: {X_scaled.std():.6f}")
print(f"Min: {X_scaled.min():.6f}")
print(f"Max: {X_scaled.max():.6f}")

class CSVDataset(Dataset):
    def __init__(self, X, y):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.int64)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        x = torch.tensor(self.X[idx])
        y = torch.tensor(self.y[idx])
        return x, y
    


    

Feature statistics after scaling:
Mean: -0.000000
Std: 1.000000
Min: -4.624010
Max: 10.784328


In [55]:
dataset = CSVDataset(X_scaled, y.values)

# Optional: Split into train/validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

BATCH_SIZE = 32

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Total batches per epoch: {len(train_loader)}")

Training samples: 2404
Validation samples: 601
Total batches per epoch: 76


In [56]:
class SimpleNet(nn.Module):  # Define a neural network class that inherits from PyTorch's nn.Module
    def __init__(self, input_size, num_classes):  # Constructor method that takes input size and number of output classes
        super().__init__()  # Call the parent class (nn.Module) constructor to initialize the neural network
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)  # Create first fully connected layer: transforms input_size features to 64 hidden units
        self.fc3 = nn.Linear(32, num_classes)  # Create second fully connected layer: transforms 64 hidden units to num_classes outputs

    def forward(self, x):  # Define the forward pass method that specifies how data flows through the network
        x = F.relu(self.fc1(x))  # Pass input through first layer, then apply ReLU activation function (removes negative values)
        x = F.relu(self.fc2(x))  # Pass through second layer with ReLU activation
        return self.fc3(x)  # Pass the result through final layer and return the final output (no activation, raw logits)

In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

input_size = X_scaled.shape[1]  # Use the scaled data's feature count
print(f"Input size: {input_size}")
num_classes = len(np.unique(y))
print(f"Number of classes: {num_classes}")

model = SimpleNet(input_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Check for any NaN in model parameters at initialization
for name, param in model.named_parameters():
    if torch.isnan(param).any():
        print(f"WARNING: NaN found in {name} at initialization!")

Using device: cpu
Input size: 37
Number of classes: 2


In [58]:
EPOCHS = 20

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    batch_count = 0
    
    for batch_idx, (x_batch, y_batch) in enumerate(train_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        
        # Check for NaN in input data
        if torch.isnan(x_batch).any() or torch.isnan(y_batch).any():
            print(f"WARNING: NaN found in batch {batch_idx}")
            continue

        optimizer.zero_grad()
        outputs = model(x_batch)
        
        # Check for NaN in outputs
        if torch.isnan(outputs).any():
            print(f"WARNING: NaN in model outputs at batch {batch_idx}")
            break
            
        loss = criterion(outputs, y_batch)
        
        # Check for NaN in loss
        if torch.isnan(loss):
            print(f"WARNING: NaN loss at batch {batch_idx}")
            print(f"Outputs: {outputs}")
            print(f"Targets: {y_batch}")
            break
            
        loss.backward()
        
        # Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        running_loss += loss.item()
        batch_count += 1

    if batch_count > 0:
        avg_loss = running_loss / batch_count
        print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {avg_loss:.4f}")
    else:
        print(f"Epoch {epoch+1}/{EPOCHS} - No valid batches processed!")
        break

Epoch 1/20 - Loss: 0.5586
Epoch 2/20 - Loss: 0.4225
Epoch 3/20 - Loss: 0.3976
Epoch 4/20 - Loss: 0.3886
Epoch 5/20 - Loss: 0.3845
Epoch 6/20 - Loss: 0.3780
Epoch 7/20 - Loss: 0.3729
Epoch 8/20 - Loss: 0.3682
Epoch 9/20 - Loss: 0.3654
Epoch 10/20 - Loss: 0.3611
Epoch 11/20 - Loss: 0.3639
Epoch 12/20 - Loss: 0.3559
Epoch 13/20 - Loss: 0.3611
Epoch 14/20 - Loss: 0.3521
Epoch 15/20 - Loss: 0.3429
Epoch 16/20 - Loss: 0.3450
Epoch 17/20 - Loss: 0.3414
Epoch 18/20 - Loss: 0.3369
Epoch 19/20 - Loss: 0.3309
Epoch 20/20 - Loss: 0.3268


In [59]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for x_batch, y_batch in val_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        outputs = model(x_batch)
        predicted = torch.argmax(outputs, dim=1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

print(f"Validation Accuracy: {correct / total:.2%}")

Validation Accuracy: 81.03%


In [60]:
torch.save(model.state_dict(), "model.pt")